# BU Alpha: Factor & Pipeline Library

## > Alpha Source #1: Buyback Announcements
### *^based on a strategy proposed by Shahram Amini and Vijay Singal in the "Are Earnings Predictable? Evidence from Equity Issues and Buyback Announcements" paper (November 21, 2018)*

**Libraries to import:**

from quantopian.pipeline import Pipeline
from quantopian.pipeline.data.eventvestor import BuybackAuthorizations

from quantopian.pipeline.factors.eventvestor import (
    BusinessDaysSinceBuybackAuth,
    BusinessDaysSincePreviousEarnings,
)

from quantopian.pipeline.data import Fundamentals

**BB (Buyback) Pipeline defined:**

In [2]:
def make_BB_pipe(context):
    BB_pipe = Pipeline()
    
    # add BB factors to BB Pipeline
    BB_pipe.add(BuybackAuthorizations.previous_date.latest, "previous_date")
    
    BB_pipe.add(BuybackAuthorizations.previous_amount.latest, "previous_amount")
    
    BB_pipe.add(BuybackAuthorizations.previous_unit.latest, "previous_unit")

    BB_pipe.add(BusinessDaysSinceBuybackAuth(), "business_days_since_buyback")

    BB_pipe.add(BusinessDaysSincePreviousEarnings(), "business_days_since_earnings")

    # add fundamental factors to BB Pipeline
    BB_pipe.add(Fundamentals.market_cap.latest, "market_cap")
    
    BB_pipe.add(Fundamentals.shares_outstanding.latest, "shares_outstanding")        
        
    # filter out share buybacks of less than 5% of shares outstanding, or dollar buybacks         
    # of less than 5% of market cap
    significant_buyback = (BuybackAuthorizations.previous_amount.latest*mil >= BB_min_pcntg*Fundamentals.market_cap.latest) if BuybackAuthorizations.previous_unit.latest == '$M' else (BuybackAuthorizations.previous_amount.latest*mil >= BB_min_pcntg*Fundamentals.shares_outstanding.latest)
        
    # filter out buybacks that occurred less than 2 days prior to the most recent                 # earnings announcement
    not_recent_buyback = BusinessDaysSinceBuybackAuth() - BusinessDaysSincePreviousEarnings() >= 2
        
    BB_pipe.set_screen(base_universe & significant_buyback & not_recent_buyback)
    
    return BB_pipe

## > Alpha Source #2: Secondary Equity Offerings (SEOs)
### *^based on a strategy proposed by Shahram Amini and Vijay Singal in the "Are Earnings Predictable? Evidence from Equity Issues and Buyback Announcements" paper (November 21, 2018)*

**Libraries to import:**

from quantopian.pipeline import Pipeline
from quantopian.pipeline.data.eventvestor import IssueEquityAnnouncements

from quantopian.pipeline.factors.eventvestor import (
    BusinessDaysSinceIssueEquityAnnouncement,
    BusinessDaysSincePreviousEarnings,
)

from quantopian.pipeline.data import Fundamentals

**SEO Pipeline defined:**

In [3]:
def make_SEO_pipe(context):
    SEO_pipe = Pipeline()
    
    # add SEO factors to SEO Pipeline
    SEO_pipe.add(IssueEquityAnnouncements.announcement_date.latest, "previous_date")
    
    SEO_pipe.add(IssueEquityAnnouncements.issue_amount.latest, "previous_amount")
    
    SEO_pipe.add(IssueEquityAnnouncements.issue_units.latest, "previous_unit")
    
    SEO_pipe.add(BusinessDaysSinceIssueEquityAnnouncement(), "business_days_since_equity_issuance")
    
    SEO_pipe.add(BusinessDaysSincePreviousEarnings(), "business_days_since_earnings")
    
    # add fundamental factors to SEO Pipeline
    SEO_pipe.add(Fundamentals.market_cap.latest, "market_cap")

    SEO_pipe.add(Fundamentals.shares_outstanding.latest, "shares_outstanding")
        
    # filter out share SEOs of less than 10% of shares outstanding, or dollar SEOs of less   
    # than 10% of market cap
    significant_SEO = (IssueEquityAnnouncements.issue_amount.latest*mil >= SEO_min_pcntg*Fundamentals.market_cap.latest) if IssueEquityAnnouncements.issue_units.latest == '$M' else (IssueEquityAnnouncements.issue_amount.latest*mil >= SEO_min_pcntg*Fundamentals.shares_outstanding.latest)
        
    # filter out SEOs that occurred less than 2 days prior to the most recent earnings   
    # announcement       
    not_recent_SEO = BusinessDaysSinceIssueEquityAnnouncement() - BusinessDaysSincePreviousEarnings() >= 2
        
    SEO_pipe.set_screen(base_universe & significant_SEO & not_recent_SEO)
    
    return SEO_pipe